For More [Info](https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook#4.-Bayesian-Optimization-with-HYPEROPT-)


## Import Statements

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
# hyperopt imports
from hyperopt import fmin    # for minimization of object functions
from hyperopt import tpe     # for surrogate model
from hyperopt import hp      # for search space optimization
from hyperopt import Trials  # to store the trial results
from hyperopt import STATUS_OK
from sklearn.metrics import  make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report   # for model evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_validate  #Perforing grid search and CV


## load the datasets

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/end2end_DS/dataSets/riskAssement_processed.csv")

In [ ]:
data.head()

,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,...,Medical_Keyword_39_yes,Medical_Keyword_40_yes,Medical_Keyword_41_yes,Medical_Keyword_42_yes,Medical_Keyword_43_yes,Medical_Keyword_44_yes,Medical_Keyword_45_yes,Medical_Keyword_46_yes,Medical_Keyword_47_yes,Medical_Keyword_48_yes
0,16,10,0.076923,1,0.641791,0.581818,0.148536,0.323008,0.028,12,...,0,0,0,0,0,0,0,0,0,0
1,0,26,0.076923,1,0.059701,0.600000,0.131799,0.272288,0.000,1,...,0,0,0,0,0,0,0,0,0,0
2,18,26,0.076923,1,0.029851,0.745455,0.288703,0.428780,0.030,9,...,0,0,0,0,0,0,0,0,0,0
3,17,10,0.487179,1,0.164179,0.672727,0.205021,0.352438,0.042,9,...,0,0,0,0,0,0,0,0,0,0
4,15,26,0.230769,1,0.417910,0.654545,0.234310,0.424046,0.027,9,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# data split
X = data.drop(['Final_Response'], axis = 1)
Y = data['Final_Response']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1, shuffle = True)

## start optimization

In [ ]:
# define search space
space = {
    "eta":               hp.uniform('eta', 0.01, 0.2),
    "gamma":             hp.uniform("gamma", 0.0, 0.5),
    "max_depth":         hp.quniform("max_depth", 1, 15,1),
    "min_child_weight":  hp.quniform('min_child_weight', 0, 10, 1),
    "reg_alpha":         hp.quniform('reg_alpha', 40,180,1),
    "reg_lambda":        hp.uniform('reg_lambda', 0,1),
    "colsample_bytree":  hp.uniform('colsample_bytree', 0.5,1),
    "n_estimators":      hp.choice("n_estimators", [100, 200, 300, 400,500,600,800])
}

In [ ]:
def objective(space):
    clf=xgb.XGBClassifier(eta=        int(space['eta']),
                    gamma =           int(space['gamma']),
                    max_depth =       int(space['max_depth']),
                    min_child_weight= int(space['min_child_weight']),
                    reg_alpha =       int(space['reg_alpha']),
                    reg_lambda =      int(space['reg_lambda']),
                    colsample_bytree= int(space['colsample_bytree']),
                    n_estimators=     int(space['n_estimators']), 
                    tree_method=      "gpu_hist", 
                    gpu_id=            0,
                    objective=        "multi:softmax",
                    eval=             "mlogloss", 
                    num_classes=      3 
    )
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation,
            early_stopping_rounds= 50, verbose=False)
    
    pred = clf.predict(X_test)
    f1Score = f1_score(y_test, pred, average= 'micro')
    print("SCORE:", f1Score)
    print(classification_report(y_test, pred))
    return {'loss': -f1Score, 'status': STATUS_OK }

In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,      #surrogate model
                        max_evals = 100,         #100 runs
                        trials = trials)

SCORE:
0.5904689736465437
              precision    recall  f1-score   support

           0       0.68      0.27      0.39      2800
           1       0.56      0.29      0.38      3611
           2       0.58      0.95      0.73      5466

    accuracy                           0.59     11877
   macro avg       0.61      0.50      0.50     11877
weighted avg       0.60      0.59      0.54     11877

SCORE:
0.4963374589542814
              precision    recall  f1-score   support

           0       0.81      0.08      0.14      2800
           1       0.44      0.07      0.12      3611
           2       0.49      0.99      0.66      5466

    accuracy                           0.50     11877
   macro avg       0.58      0.38      0.31     11877
weighted avg       0.55      0.50      0.37     11877

SCORE:
0.5583059695209228
              precision    recall  f1-score   support

           0       0.72      0.18      0.28      2800
           1       0.53      0.23      0.32      36

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [ ]:
# params = {'colsample_bytree':    [0.7,0.8,0.9], 
#           'eta':                 [0.15,0.18,0.20], 
#           'gamma':               [0.3,0.4,0.5], 
#           'max_depth':           [13,14,15], 
#           'min_child_weight':    [0.9,1,1.5],
#           'n_estimators':        [6,8,10],
#           'reg_alpha':           [40,45,50], 
#           'reg_lambda':          [0.02,0.05,0.08]}

params = {'colsample_bytree':    [0.9], 
          'eta':                 [0.17], 
          'gamma':               [0.3], 
          'max_depth':           [15], 
          'min_child_weight':    [1],
          'n_estimators':        [8],
          'reg_alpha':           [40,45,50], 
          'reg_lambda':          [0.01,0.05,0.1]}

In [ ]:
xgbc = xgb.XGBClassifier(tree_method=      "gpu_hist", 
                              gpu_id=       0,
                           objective=      "multi:softmax",
                                eval=      "mlogloss",
                         num_classes=      3,
              early_stopping_rounds=       10
                         )

scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'micro'),
           'recall': make_scorer(recall_score, average = 'micro'),
           'f1': make_scorer(f1_score, average = 'micro')}

# create 3 folds
folds = StratifiedKFold(n_splits= 3, shuffle= True, random_state= 42)
#Gridsearch for hyperparam tuning
model= GridSearchCV(estimator= xgbc, param_grid= params, cv= folds, return_train_score= True, scoring = scoring, refit='f1', verbose= 1000)


In [ ]:
model.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3; 1/9] START colsample_bytree=0.9, eta=0.17, gamma=0.3, max_depth=15, min_child_weight=1, n_estimators=8, reg_alpha=40, reg_lambda=0.01
[CV 1/3; 1/9] END colsample_bytree=0.9, eta=0.17, gamma=0.3, max_depth=15, min_child_weight=1, n_estimators=8, reg_alpha=40, reg_lambda=0.01; accuracy: (train=0.657, test=0.635) f1: (train=0.657, test=0.635) precision: (train=0.657, test=0.635) recall: (train=0.657, test=0.635) total time=   1.6s
[CV 2/3; 1/9] START colsample_bytree=0.9, eta=0.17, gamma=0.3, max_depth=15, min_child_weight=1, n_estimators=8, reg_alpha=40, reg_lambda=0.01
[CV 2/3; 1/9] END colsample_bytree=0.9, eta=0.17, gamma=0.3, max_depth=15, min_child_weight=1, n_estimators=8, reg_alpha=40, reg_lambda=0.01; accuracy: (train=0.655, test=0.636) f1: (train=0.655, test=0.636) precision: (train=0.655, test=0.636) recall: (train=0.655, test=0.636) total time=   1.5s
[CV 3/3; 1/9] START colsample_bytree=0.9, eta=0.17, gamma=

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
             estimator=XGBClassifier(early_stopping_rounds=10, eval='mlogloss',
                                     gpu_id=0, num_classes=3,
                                     objective='multi:softmax',
                                     tree_method='gpu_hist'),
             param_grid={'colsample_bytree': [0.9], 'eta': [0.17],
                         'gamma': [0.3], 'max_depth': [15],
                         'min_child_weight': [1], 'n_estimators': [8],
                         'reg_alpha': [40, 45, 50],
                         'reg_lambda': [0.01, 0.05, 0.1]},
             refit='f1', return_train_score=True,
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score, average=micro),
                      'precision': make_scorer(precision_score, average=micro),
                      'recall': make_scorer(recall_score, average=micro)},
             

In [ ]:
print("Best parameters:", model.best_params_)

Best parameters: {'colsample_bytree': 0.9, 'eta': 0.17, 'gamma': 0.3, 'max_depth': 15, 'min_child_weight': 1, 'n_estimators': 8, 'reg_alpha': 45, 'reg_lambda': 0.05}


In [ ]:
clf = xgb.XGBClassifier(colsample_bytree= 0.9,
                                     eta= 0.17,
                                  mgamma= 0.3,
                               max_depth= 15,
                        min_child_weight= 1,
                            n_estimators= 8,
                               reg_alpha= 45,
                              reg_lambda= 0.05,
                             tree_method= "gpu_hist", 
                                  gpu_id= 0,
                               objective= "multi:softmax",
                                    eval= "mlogloss",
                             num_classes= 3
                         )

In [ ]:
clf.fit(X_train, y_train)


XGBClassifier(colsample_bytree=0.9, eta=0.17, eval='mlogloss', gpu_id=0,
              max_depth=15, mgamma=0.3, n_estimators=8, num_classes=3,
              objective='multi:softprob', reg_alpha=45, reg_lambda=0.05,
              tree_method='gpu_hist')

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.37      0.47      2800
           1       0.58      0.42      0.49      3611
           2       0.66      0.91      0.76      5466

    accuracy                           0.63     11877
   macro avg       0.62      0.57      0.57     11877
weighted avg       0.62      0.63      0.61     11877

